In [2]:
import cv2
import numpy as np

background = None
accumulated_weight = 0.5

ROI_top = 100
ROI_bottom = 300
ROI_right = 150
ROI_left = 350

In [3]:
def cal_accum_avg(frame, accumulated_weight=0.5):

    global background
    
    if background is None:
        background = frame.copy().astype("float")
        return None

    cv2.accumulateWeighted(frame, background, accumulated_weight)

In [4]:
def segment_hand(frame, threshold=25):
    global background
    
    diff = cv2.absdiff(background.astype("uint8"), frame)
    cv2.imshow("difference", diff)
    _ , thresholded = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)
    
    # Grab the external contours for the image
    contours, hierarchy = cv2.findContours(thresholded.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) == 0:
        return None
    else:
        
        hand_segment_max_cont = max(contours, key=cv2.contourArea)
        
        return (thresholded, hand_segment_max_cont)

In [67]:
cam = cv2.VideoCapture(0)

num_frames = 0
element = "M"
num_imgs_taken = 0
try:
#making directory
    import os
    contour_test_dir = "D:\\Desktop\\ML\\sign_language\\gesture1\\contours\\test"
    contour_train_dir = "D:\\Desktop\\ML\\sign_language\\gesture1\\contours\\train"
    gray_test_dir = "D:\\Desktop\\ML\\sign_language\\gesture1\\edge\\gray\\test"
    gray_train_dir = "D:\\Desktop\\ML\\sign_language\\gesture1\\edge\\gray\\train"
    white_test_dir = "D:\\Desktop\\ML\\sign_language\\gesture1\\edge\\white\\test"
    white_train_dir = "D:\\Desktop\\ML\\sign_language\\gesture1\\edge\\white\\train"
    contour_test_path = os.path.join(contour_test_dir, element)
    contour_train_path = os.path.join(contour_train_dir, element)
    gray_test_path = os.path.join(gray_test_dir, element)
    gray_train_path = os.path.join(gray_train_dir, element)
    white_test_path = os.path.join(white_test_dir, element)
    white_train_path = os.path.join(white_train_dir, element)


    os.mkdir(contour_test_path)
    os.mkdir(contour_train_path)
    os.mkdir(gray_test_path)
    os.mkdir(gray_train_path)
    os.mkdir(white_test_path)
    os.mkdir(white_train_path)
except: 
    pass


while True:
    ret, frame = cam.read()

    # filpping the frame to prevent inverted image of captured frame...
    frame = cv2.flip(frame, 1)

    frame_copy = frame.copy()

    roi = frame[ROI_top:ROI_bottom, ROI_right:ROI_left]

    gray_frame = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    gray_frame = cv2.GaussianBlur(gray_frame, (9, 9), 0)
    canny_gray = cv2.Canny(gray_frame, 100,200, apertureSize=3, L2gradient=True)
    canny_white = cv2.Canny(roi, 100,125)

    if num_frames < 60:
        cal_accum_avg(gray_frame, accumulated_weight)
        if num_frames <= 59:
            
            cv2.putText(frame_copy, "FETCHING BACKGROUND...PLEASE WAIT", (80, 400), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2)
            #cv2.imshow("Sign Detection",frame_copy)
         
    #Time to configure the hand specifically into the ROI...
    elif num_frames <= 200: 

        hand = segment_hand(gray_frame)
        
        cv2.putText(frame_copy, "Adjust hand Gesture for " + str(element), (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
        
        # Checking if hand is actually detected by counting number of contours detected...
        if hand is not None:
            
            thresholded, hand_segment = hand

            # Draw contours around hand segment
            cv2.drawContours(frame_copy, [hand_segment + (ROI_right, ROI_top)], -1, (255, 0, 0),1)
            cv2.drawContours(gray_frame, hand_segment, -1, (255, 0, 0),1)
            canny_gray = cv2.Canny(gray_frame, 100, 200,  apertureSize=3, L2gradient=True)
            canny_white = cv2.Canny(roi, 100, 125)
            
            cv2.putText(frame_copy, str(num_frames)+" images for " + str(element), (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

            # Also display the thresholded image
            cv2.imshow("Thresholded Hand Image", thresholded)
    
    else: 
        
        # Segmenting the hand region...
        hand = segment_hand(gray_frame)
        
        # Checking if we are able to detect the hand...
        if hand is not None:
            
            # unpack the thresholded img and the max_contour...
            thresholded, hand_segment = hand

            # Drawing contours around hand segment
            cv2.drawContours(frame_copy, [hand_segment + (ROI_right, ROI_top)], -1, (255, 0, 0),1)
            cv2.drawContours(gray_frame, hand_segment, -1, (255, 0, 0),1)
            canny_gray = cv2.Canny(gray_frame, 100,200, apertureSize=3,L2gradient=True)
            canny_white = cv2.Canny(roi, 100,125)
            
            cv2.putText(frame_copy, str(num_frames), (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            #cv2.putText(frame_copy, str(num_frames)+"For" + str(element), (70, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            cv2.putText(frame_copy, str(num_imgs_taken) + 'images' +"For" + str(element), (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            
            # Displaying the thresholded image
            cv2.imshow("Thresholded Hand Image", thresholded)
            if num_imgs_taken <= 300:
                if num_imgs_taken<=40:
#                     pass
                    cv2.imwrite(r"D:\\Desktop\ML\\sign_language\\gesture1\\contours\\test\\"+str(element)+"\\" + str(num_imgs_taken) + '.jpg', thresholded)
                    cv2.imwrite(r"D:\\Desktop\ML\\sign_language\\gesture1\\edge\\white\\test\\"+str(element)+"\\" + str(num_imgs_taken) + '.jpg', canny_white)
                    cv2.imwrite(r"D:\\Desktop\ML\\sign_language\\gesture1\\edge\\gray\\test\\"+str(element)+"\\" + str(num_imgs_taken) + '.jpg', canny_gray)
                cv2.imwrite(r"D:\\Desktop\ML\\sign_language\\gesture1\\contours\\train\\"+str(element)+"\\" + str(num_imgs_taken) + '.jpg', thresholded)
                cv2.imwrite(r"D:\\Desktop\ML\\sign_language\\gesture1\\edge\\white\\train\\"+str(element)+"\\" + str(num_imgs_taken) + '.jpg', canny_white)
                cv2.imwrite(r"D:\\Desktop\ML\\sign_language\\gesture1\\edge\\gray\\train\\"+str(element)+"\\" + str(num_imgs_taken) + '.jpg', canny_gray)
        
#                 cv2.imwrite(r"D:\Desktop\ML\sign_language\gesture"+"\\" + str(num_imgs_taken) + '.jpg', thresholded)
            else:
                break
            num_imgs_taken +=1
        else:
            cv2.putText(frame_copy, 'No hand detected...', (200, 400), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

    # Drawing ROI on frame copy
    cv2.rectangle(frame_copy, (ROI_left, ROI_top), (ROI_right, ROI_bottom), (255,128,0), 3)
    
    cv2.putText(frame_copy, "DataFlair hand sign recognition_ _ _", (10, 20), cv2.FONT_ITALIC, 0.5, (51,255,51), 1)
    
    # increment the number of frames for tracking
    num_frames += 1

    # Display the frame with segmented hand
    cv2.imshow("Sign Detection", frame_copy)
    cv2.imshow("gray_frame", gray_frame)
    cv2.imshow("canny_gray", canny_gray)
    cv2.imshow("canny_white", canny_white)

    # Closing windows with Esc key...(any other key with ord can be used too.)
    k = cv2.waitKey(1) & 0xFF

    if k == 27:
        break

In [68]:
# Releasing camera & destroying all the windows...

cv2.destroyAllWindows()
cam.release()